# CrewAI Workflow: E-commerce Price Monitoring with Groq LLM

This notebook demonstrates a simple single-agent workflow using CrewAI:
- Price Intelligence Analyst searches for products and identifies the best deals

It showcases how to:
- Configure an LLM provider (Groq) for CrewAI
- Configure a website search tool (Amazon-focused) with an embedder
- Define an Agent with role, goal, and backstory
- Create a parameterized Task and execute the crew
- Render results in the notebook

Prerequisites:
- A valid `GROQ_API_KEY` available as an environment variable
- Python 3.10+ and the packages: `crewai`, `groq`, `crewai-tools` (versions per your environment)

Tip: If running in a new environment, install deps first in a separate cell.


## Setup and Dependencies

Install the required packages (skip if already installed):

```python
# !pip install crewai groq crewai-tools
```

Then import the core classes and tools you'll use from CrewAI.


In [ ]:
# !pip install crewai groq crewai-tools

## Configure the LLM (Groq)

This notebook uses Groq's hosted LLM via CrewAI.
- Ensure `GROQ_API_KEY` is set in your environment (Windows: `setx GROQ_API_KEY "<your-key>"`, macOS/Linux: `export GROQ_API_KEY=<your-key>` then restart your shell).
- Choose a supported model string (e.g., `groq/llama-3.3-70b-versatile`).

If the key is missing, CrewAI will raise an authentication error when the agent runs.


In [ ]:
# Core CrewAI building blocks and a website search tool
from crewai import Agent, Task, Crew, LLM
from crewai_tools import WebsiteSearchTool

## Configure the Website Search Tool (Amazon)

We specialize the `WebsiteSearchTool` to `amazon.com` and set both the LLM provider and the embedder:
- LLM: Groq model used for reasoning and extraction
- Embedder: local `ollama` endpoint (`nomic-embed-text`) for vectorization


In [ ]:
# Instantiate the shared LLM used by the agent and tool
llm = LLM(model="groq/llama-3.3-70b-versatile")

In [ ]:
# E-commerce focused search tool
# Configures an Amazon-specific retriever with Groq LLM and a local Ollama embedder
price_monitor_tool = WebsiteSearchTool(
    website="https://amazon.com",
    config=dict(
        llm=dict(
            provider="groq",
            config=dict(
                model="llama-3.3-70b-versatile",
                temperature=0.2
            ),
        ),
        embedder=dict(
            provider="ollama",
            config=dict(
                model="nomic-embed-text:latest",
                base_url="http://localhost:11434"
            ),
        ),
    ),
)

## Define Agent and Task

We define one agent with a clear role and goal, then create a single task parameterized by category and budget.


In [ ]:
# Define the agent persona and connect the tool and LLM
price_agent = Agent(
    role="Price Intelligence Analyst",
    goal="Monitor product prices and find best deals across e-commerce platforms",
    backstory="You're a savvy shopping expert who tracks price trends and identifies bargains.",
    tools=[price_monitor_tool],  # grants the agent the ability to search and parse Amazon pages
    llm=llm,
    verbose=True
)

# Create a task template parameterized by product category and budget
price_task = Task(
    description="Find the best deals on {product_category} under ${budget}",
    expected_output="List of top 5 products with prices, ratings, and deal alerts",
    agent=price_agent
)

## Run and Render Results

Execute the crew with `crew.kickoff()` and then render the final output. In this workflow, the output is the list of best deals identified by the agent.


In [ ]:
# Assemble the crew with our single agent and run the workflow
crew = Crew(
    agents=[price_agent],
    tasks=[price_task],
    verbose=True
)

# Provide runtime inputs for the templated task
result = crew.kickoff(inputs={"product_category": "Electronics", "budget": 300})

### Present the results
We render the `result` as Markdown for readability in the notebook.


In [ ]:
# Display the result in formatted markdown for better readability
from IPython.display import Markdown, display

# Convert result (which may be a structured object) to string and display
# Using Markdown ensures nicer rendering in the notebook UI
display(Markdown(str(result)))